In [1]:
using BenchmarkTools
using Plots
using DRR
using Zygote

┌ Info: Precompiling DRR [b7cdf4c7-cd72-4265-95a4-eec1a441a985]
└ @ Base loading.jl:1423


In [2]:
# Read the volume
volume, ΔX, ΔY, ΔZ = read_dicom("../data/cxr"; pad=true)
# volume = volume[:, :, 2:end-1]
grid, pixels = volume2grid(volume, ΔX, ΔY, ΔZ)

# Define the camera
center = Vec3(180., 180., -100)
camera = Camera(center)

# Define the detector plane
center = Vec3(180., 180., 500.)
normal = Vec3(1., -1., -1.)
height, width = 601, 601
Δx, Δy = 2., 2.
detector = Detector(center, normal, height, width, Δx, Δy)

Detector(Vec3{Float64}(180.0, 180.0, 500.0), Vec3{Float64}(1.0, -1.0, -1.0), 601, 601, 2.0, 2.0)

In [3]:
spacing = 0.02

0.02

In [ ]:
gradient(spc -> make_drr(grid, pixels, camera, detector, spc), spacing)

In [4]:
drr = make_drr(grid, volume, camera, detector, spacing)
heatmap(drr, c=:grays)

LoadError: type Array has no field cutPoints

In [5]:
function raytrace_trilinear(ray, spacing::Float64, grid, pixels)
    pts = trace.(0:spacing:1; ray=ray)
    interpolations = interpolate.(pts; grid, pixels)
    return sum(interpolations) / length(pts)
end
    

raytrace_trilinear (generic function with 1 method)

In [7]:
findz(x, y; a, b, c, d) = (d - a * x - b * y) / c
findz(x::Tuple{Float64,Float64}; a, b, c, d) = findz(x[1], x[2]; a, b, c, d)
append(xy, z) = Vec3(xy..., z)

# Construct the detector array
function make_plane(detector::Detector)
    d = dotprod(detector.center, detector.normal)
    xs = (-detector.height÷2:1:detector.height÷2) * detector.Δx
    ys = (-detector.width÷2:1:detector.width÷2) * detector.Δy
    xys = product(xs, ys) |> collect
    zs = findz.(xys; a=detector.normal.x, b=detector.normal.y, c=detector.normal.z, d=d)
    return append.(xys, zs)
end
get_rays(camera, plane) = [Ray(camera.center, pixel - camera.center) for pixel in plane]

import Base: product
# Set up the detector plane
plane = make_plane(detector)
projector = get_rays(camera, plane);

In [9]:
projector[1]

DRR.Ray_{Float64}(Vec3{Float64}(180.0, 180.0, -100.0), Vec3{Float64}(-780.0, -780.0, 600.0), 1255.7069721873809)

In [15]:
gradient(spc -> raytrace_trilinear(projector[1000], spc, grid, pixels), spacing)

LoadError: Mutating arrays is not supported -- called setindex!(::Vector{Int64}, _...)

In [16]:
W = rand(2, 3); x = rand(3);

gradient(W -> sum(W*x), W)

([0.8935398899772946 0.23748846424555048 0.7752539057922665; 0.8935398899772946 0.23748846424555048 0.7752539057922665],)

In [14]:
raytrace_trilinear(projector[1000], spacing, grid, pixels)

-723.1175208586938

In [17]:
using GridInterpolations

In [18]:
grid = RectangleGrid([0., 0.5, 1.],[0., 0.5, 1.])  	# rectangular grid
sGrid = SimplexGrid([0., 0.5, 1.],[0., 0.5, 1.])	# simplex grid
gridData = [8., 1., 6., 3., 5., 7., 4., 9., 2.]   	# vector of value data at each cut
x = [0.25, 0.75]  

interpolate(grid,gridData,x)

5.25

In [19]:
gradient(x -> interpolate(grid,gridData,x), x)

LoadError: Mutating arrays is not supported -- called copyto!(::Vector{Float64}, _...)